In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import sys
sys.path.append('../src')
import numpy as np
import seaborn as sns
import torch
import matplotlib
from matplotlib import pyplot as plt
from sklearn import metrics
import sklearn.calibration
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
# plt.style.use('dark_background')
import data
from skorch.callbacks import Checkpoint
from skorch import NeuralNetRegressor
from config import *
from tqdm import tqdm
import pickle as pkl
import train_reg
from math import floor
from copy import deepcopy
from sklearn.metrics import confusion_matrix
import config
import models
import pandas as pd
import features
from scipy.stats import skew, pearsonr
import outcomes
import neural_networks
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.nn import functional as F
import interpret
import matplotlib.pyplot as plt
from functools import partial
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
import viz

# prepare data

In [2]:
# currently trained only on 'clath_aux+gak_a7d2_new'
# dsets = ['clath_aux+gak_new']
dsets = ['clath_aux_dynamin']
splits = ['test']
# feat_names = ['X_same_length_normalized'] # + data.select_final_feats(data.get_feature_names(df))

#feat_names = ['X_same_length_normalized'] + data.select_final_feats(data.get_feature_names(df))
              #['mean_total_displacement', 'mean_square_displacement', 'lifetime']
meta = ['cell_num', 'Y_sig_mean', 'Y_sig_mean_normalized']
dfs, feat_names = data.load_dfs_for_lstm(dsets=dsets, splits=splits, meta=meta, normalize=False)
print('type', type(dfs))
# load model
p = 1
# load data
# df = dfs[('clath_aux+gak_a7d2', 'train')]
# df = dfs[('clath_aux+gak_new', 'test')]
df = dfs[list(dfs.keys())[0]]
X = df[feat_names[:p]]
y = df['y_consec_thresh']

100%|██████████| 1/1 [00:15<00:00, 15.76s/it]

type <class 'dict'>


# look at different dsets

In [3]:
print("loading data")
dsets = ['clath_aux+gak_a7d2', 'clath_aux+gak', 'clath_aux+gak_a7d2_new', 'clath_aux+gak_new', 'clath_gak', 'clath_aux_dynamin'] # need to add dynamin
splits = ['train', 'test']
#feat_names = ['X_same_length_normalized'] + data.select_final_feats(data.get_feature_names(df))
              #['mean_total_displacement', 'mean_square_displacement', 'lifetime']
meta = ['cell_num', 'Y_sig_mean', 'Y_sig_mean_normalized', 'y_consec_thresh']
dfs, feat_names = data.load_dfs_for_lstm(dsets=dsets, splits=splits, meta=meta, filter_short=False, normalize=False)

# df_full = pd.concat([dfs[(k, s)]
#                  for (k, s) in dfs
#                  if s == 'train'])[feat_names + meta]
# df_full = df_full.dropna()
ds = {(k, v): dfs[(k, v)]
      for (k, v) in sorted(dfs.keys(), key=lambda x: x[1] + x[0])
      #if not k == 'clath_aux+gak_a7d2_new'
     }

  0%|          | 0/6 [00:00<?, ?it/s]

loading data


100%|██████████| 6/6 [02:30<00:00, 25.08s/it]


**summarize dsets**

In [4]:
vals = {} # key is dataset, Partition
for k in sorted(ds, key=lambda kv: (kv[0], kv[1])):
    d = ds[k]
    d.short = d.lifetime <= 15
    # print(k, d.lifetime.min())
    y = d['y_consec_thresh']
    feats = ['Total clean tracks',
             'Difficult tracks', 'Difficult valid events', 
             'Short tracks', 'Short valid events',]
    vals[k] = [d.shape[0],
            (~d.short).sum(), y[~d.short].sum(),
            d.short.sum(), y[d.short].sum()
           ]
    vals[k] = [int(x) for x in vals[k]]
#     print(feats, vals)
df = pd.DataFrame.from_dict(vals).transpose().round(decimals=0)
df.columns = feats
df.index = df.index.set_names(['Dataset', 'Partition'])
print(df.to_latex(index=True, index_names=True))

\begin{tabular}{llrrrrr}
\toprule
          &       &  Total clean tracks &  Difficult tracks &  Difficult valid events &  Short tracks &  Short valid events \\
Dataset & Partition &                     &                   &                         &               &                     \\
\midrule
clath\_aux+gak & test &                1551 &               494 &                     264 &          1057 &                  96 \\
          & train &                5783 &              1750 &                    1054 &          4033 &                 464 \\
clath\_aux+gak\_a7d2 & test &                2448 &              1018 &                     439 &          1430 &                  73 \\
          & train &                7245 &              2979 &                    1523 &          4266 &                 212 \\
clath\_aux+gak\_a7d2\_new & test &               16893 &              1495 &                     978 &         15398 &                2157 \\
          & train &               497